# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [206]:
from nltk.util import ngrams

##  Часть 3. Нейронная сеть


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor

### Предварительная обработка данных

#### Загрузим и почистим имена

In [207]:
with open("female.txt") as f:
    fnames = f.readlines()
    fnames = [ name.strip() for name in fnames ]

In [208]:
with open("male.txt") as f:
    mnames = f.readlines()
    mnames = [ name.strip() for name in mnames ]

In [209]:
print(fnames[:10])
print('fnames len: ', len(fnames))
print(mnames[:10])
print('mnames len: ', len(mnames))

['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi', 'Abbie', 'Abby', 'Abigael', 'Abigail', 'Abigale']
fnames len:  5001
['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot', 'Abbott', 'Abby', 'Abdel', 'Abdul', 'Abdulkarim']
mnames len:  2943


In [210]:
fnames_cleared = [x for x in fnames if x not in mnames]
np.random.shuffle(fnames_cleared)

In [211]:
mnames_cleared = [x for x in mnames if x not in fnames]
np.random.shuffle(mnames_cleared)

In [212]:
print(fnames_cleared[:10])
print('fnames len: ', len(fnames_cleared))
print(mnames_cleared[:10])
print('mnames len: ', len(mnames_cleared))

['Nessie', 'Anetta', 'Audi', 'Dorry', 'Ardis', 'Lillis', 'Kelli', 'Harli', 'Anna-Diane', 'Simonne']
fnames len:  4635
['Sanford', 'Parnell', 'Giff', 'Barth', 'Giraldo', 'Teador', 'Pepillo', 'Kingsly', 'Cesar', 'Dougie']
mnames len:  2578


In [213]:
coef = int(len(mnames_cleared) * 0.8)
print(coef)

2062


In [214]:
import numpy as np
fzeros = np.zeros(len(fnames_cleared))
mzeros = np.ones(len(mnames_cleared))
print(len(fzeros))
print(len(mzeros))

4635
2578


Объединим мужские и женские имена, одновременно присвоив им метку пола

In [215]:
fnames_train = np.vstack((fnames_cleared[:coef], fzeros[:coef])).T
print(fnames_train[:10])
print(fnames_train.shape)
fnames_test = np.vstack((fnames_cleared[coef : ], fzeros[coef : ])).T
print(fnames_test[:10])
print(fnames_test.shape)

[['Nessie' '0.0']
 ['Anetta' '0.0']
 ['Audi' '0.0']
 ['Dorry' '0.0']
 ['Ardis' '0.0']
 ['Lillis' '0.0']
 ['Kelli' '0.0']
 ['Harli' '0.0']
 ['Anna-Diane' '0.0']
 ['Simonne' '0.0']]
(2062, 2)
[['Amargo' '0.0']
 ['Alia' '0.0']
 ['Casie' '0.0']
 ['Blinny' '0.0']
 ['Roxane' '0.0']
 ['Joyan' '0.0']
 ['Sharyl' '0.0']
 ['Angelique' '0.0']
 ['Cicily' '0.0']
 ['Farand' '0.0']]
(2573, 2)


In [216]:
mnames_train = np.vstack((mnames_cleared[:coef], mzeros[:coef])).T
print(fnames_train[:10])
print(fnames_train.shape)
mnames_test = np.vstack((mnames_cleared[coef : ], mzeros[coef : ])).T
print(mnames_test[-10:])
print(mnames_test.shape)

[['Nessie' '0.0']
 ['Anetta' '0.0']
 ['Audi' '0.0']
 ['Dorry' '0.0']
 ['Ardis' '0.0']
 ['Lillis' '0.0']
 ['Kelli' '0.0']
 ['Harli' '0.0']
 ['Anna-Diane' '0.0']
 ['Simonne' '0.0']]
(2062, 2)
[['Ambrosius' '1.0']
 ['Hershel' '1.0']
 ['Sander' '1.0']
 ['Renaud' '1.0']
 ['Elwyn' '1.0']
 ['Walt' '1.0']
 ['Andrzej' '1.0']
 ['Robb' '1.0']
 ['Engelbart' '1.0']
 ['Cyrille' '1.0']]
(516, 2)


In [217]:
X_train = np.append(fnames_train, mnames_train, axis=0)
X_test = np.append(fnames_test, mnames_test, axis=0)
print(X_train[coef - 10 : coef + 10])
print(X_train.shape)

[['Tomasina' '0.0']
 ['Consuela' '0.0']
 ['Maryanna' '0.0']
 ['Mellisa' '0.0']
 ['Donella' '0.0']
 ['Gwenny' '0.0']
 ['Jacquette' '0.0']
 ['Darleen' '0.0']
 ['Cathryn' '0.0']
 ['Tiertza' '0.0']
 ['Sanford' '1.0']
 ['Parnell' '1.0']
 ['Giff' '1.0']
 ['Barth' '1.0']
 ['Giraldo' '1.0']
 ['Teador' '1.0']
 ['Pepillo' '1.0']
 ['Kingsly' '1.0']
 ['Cesar' '1.0']
 ['Dougie' '1.0']]
(4124, 2)


Еще раз перемешиваем все имена

In [218]:
np.random.shuffle(X_train)
np.random.shuffle(X_test)

In [219]:
print(X_train[coef - 10 : coef + 10])

[['Camile' '0.0']
 ['Leia' '0.0']
 ['Spiro' '1.0']
 ['Laurella' '0.0']
 ['Daniella' '0.0']
 ['Stephani' '0.0']
 ['Pieter' '1.0']
 ['Anjela' '0.0']
 ['Berta' '0.0']
 ['Edouard' '1.0']
 ['Ben' '1.0']
 ['Cornelius' '1.0']
 ['Ruben' '1.0']
 ['Fletch' '1.0']
 ['Xenos' '1.0']
 ['Cosmo' '1.0']
 ['Claresta' '0.0']
 ['Archy' '1.0']
 ['Marchall' '1.0']
 ['Vanni' '0.0']]


Функция вывода метрик

In [220]:
from sklearn.metrics import *
def metrics_test(predicted, target):
    acc = accuracy_score(predicted, target)
    micro_f1 = f1_score(predicted, target, average = 'micro')
    micro_p = f1_score(predicted, target, average = 'micro')
    micro_r = f1_score(predicted, target, average = 'micro')
    macro_f1 = f1_score(predicted, target, average = 'macro')
    macro_p = f1_score(predicted, target, average = 'macro')
    macro_r = f1_score(predicted, target, average = 'macro')
    print('acc={0:1.4f}'.format(acc))
    print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}'.format(micro_f1, micro_p, micro_r))
    print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}\n'.format(macro_f1, macro_p, macro_r))

### Базовый метод классификации

In [221]:
from nltk.util import ngrams

In [222]:
X_train[:, 0]

array(['Derron', 'Boyd', 'Tre', ..., 'Lincoln', 'Greggory', 'Pierrette'],
      dtype='<U32')

Создадим массив содержащий в строках имена в виде биграмм (по аналогии с предложениями или целыми текстами. Только в текстах содержались слова, а мы будем работать, как будто бы Имя - Предложение, Биграмма - Слово в предложении)

In [270]:
from nltk import ngrams
def get_ngrams(train, ngrams_count = 2):
    sequences = []
    for words, sex in train[:]:
        seq = list(words.lower())
        bigrams = ngrams(seq, ngrams_count)
        bi = []
        for grams in bigrams:           
            if len(grams) >= ngrams_count:
#                 print("{} : n {}".format(len(grams), ngrams_count))
                bi.append("".join(grams))
        sequences.append([" ".join(bi), int(float(sex))])
    return np.array(sequences)    

### Посмотрим на предсказания используя Биграммы

In [271]:
X_train_gram = get_ngrams(X_train)
X_test_gram = get_ngrams(X_test)

In [272]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_gram[:,0])
X_train_counts.shape

(4124, 425)

In [273]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, X_train_gram[:,1])

In [274]:
X_test_fin = count_vect.transform(X_test_gram[:,0])
predicted = clf.predict(X_test_fin)

In [275]:
metrics_test(predicted, X_test_gram[:,1])

acc=0.7614
micro F1=0.7614, micro P=0.7614, micro R=0.7614
macro F1=0.6709, macro P=0.6709, macro R=0.6709



### Посмотрим на предсказания используя Три-граммы

In [276]:
X_train_gram = get_ngrams(X_train, ngrams_count=3)
X_test_gram = get_ngrams(X_test, ngrams_count=3)

In [277]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_gram[:,0])
X_train_counts.shape

(4124, 2680)

In [278]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, X_train_gram[:,1])

In [279]:
X_test_fin = count_vect.transform(X_test_gram[:,0])
predicted = clf.predict(X_test_fin)

In [280]:
metrics_test(predicted, X_test_gram[:,1])

acc=0.8012
micro F1=0.8012, micro P=0.8012, micro R=0.8012
macro F1=0.7144, macro P=0.7144, macro R=0.7144



### Посмотрим на предсказания используя Четыре-граммы

In [251]:
X_train_gram = get_ngrams(X_train, ngrams_count=4)
X_test_gram = get_ngrams(X_test, ngrams_count=4)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [252]:
X_train_gram

array([['derr erro rron', '1'],
       ['boyd', '1'],
       ['', '1'],
       ..., 
       ['linc inco ncol coln', '1'],
       ['greg regg eggo ggor gory', '1'],
       ['pier ierr erre rret rett ette', '0']],
      dtype='<U59')

In [253]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_gram[:,0])
X_train_counts.shape

(4124, 5738)

In [254]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, X_train_gram[:,1])

In [255]:
X_test_fin = count_vect.transform(X_test_gram[:,0])
predicted = clf.predict(X_test_fin)

In [256]:
metrics_test(predicted, X_test_gram[:,1])

acc=0.8249
micro F1=0.8249, micro P=0.8249, micro R=0.8249
macro F1=0.7225, macro P=0.7225, macro R=0.7225



### RNN

Посчитаем максимальные значения для того что бы понять размерности входных данных

In [310]:
totalEntries = X_train.shape[0]
maxlen = len(max( X_train[:, 0] , key=len)) 

Теперь создадим таблицы для перевода символов в индексы и обратно

In [311]:
chars = set(  "".join(X_train[:, 0])  )
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Посмотрим на размерности

In [312]:
print("total endtries " , totalEntries)
print("max len " , maxlen)
print('total chars:', len(chars))

total endtries  4124
max len  15
total chars: 54


Подготовим пустые массивы нужной нам размерности что бы после заполнить их данными

In [313]:
X = np.zeros((totalEntries , maxlen, len(chars) ), dtype=np.bool)
y = np.zeros((totalEntries , 2 ), dtype=np.bool)

Заполним данными подготовленные массивы

In [314]:
X_train

array([['Derron', '1.0'],
       ['Boyd', '1.0'],
       ['Tre', '1.0'],
       ..., 
       ['Lincoln', '1.0'],
       ['Greggory', '1.0'],
       ['Pierrette', '0.0']],
      dtype='<U32')

Вот тут у нас фактически one_hot_encoding. Я взял с примера, хотя есть подозрение что можно просто прикрутить стандартный, предварительно разбив все придложения (в нашем случае имена) на массивы символов и скормить стандартному (хоть из sklearn хоть из keras)

In [315]:
for i, (name, sex) in enumerate(X_train):
    for t, char in enumerate(name):
        X[i, t, char_indices[char]] = 1
    y[i, int(float(sex))] = 1

Создадим модель сети

In [317]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam')

Using TensorFlow backend.


In [320]:
model.fit(X, y, batch_size=16, epochs=20)
model.save_weights('my_model_weights.h5')
score = model.evaluate(X, y, batch_size=16)
print("score " , score)

Epoch 1/20
4124/4124 [==============================] - 49s 12ms/step - loss: 0.5829
Epoch 2/20
4124/4124 [==============================] - 51s 12ms/step - loss: 0.5141
Epoch 3/20
4124/4124 [==============================] - 50s 12ms/step - loss: 0.4890
Epoch 4/20
4124/4124 [==============================] - 49s 12ms/step - loss: 0.4692
Epoch 5/20
4124/4124 [==============================] - 50s 12ms/step - loss: 0.4384
Epoch 6/20
4124/4124 [==============================] - 51s 12ms/step - loss: 0.4126
Epoch 7/20
4124/4124 [==============================] - 52s 13ms/step - loss: 0.3905
Epoch 8/20
4124/4124 [==============================] - 50s 12ms/step - loss: 0.3789
Epoch 9/20
4124/4124 [==============================] - 50s 12ms/step - loss: 0.3507
Epoch 10/20
4124/4124 [==============================] - 49s 12ms/step - loss: 0.3319
Epoch 11/20
4124/4124 [==============================] - 52s 13ms/step - loss: 0.3249
Epoch 12/20
4124/4124 [==============================] - 49s 12

Посчитаем максимальные значения для того что бы понять размерности входных данных

In [323]:
totalEntries_t = X_test.shape[0]
maxlen_t = len(max( X_test[:, 0] , key=len)) 

Подготовим пустые массивы нужной нам размерности что бы после заполнить их данными

In [326]:
X_t = np.zeros((totalEntries_t , maxlen_t, len(chars) ), dtype=np.bool)
y_t = np.zeros((totalEntries_t , 2 ), dtype=np.bool)

Подготовим тестовые данные

In [328]:
for i, (name, sex) in enumerate(X_test):
    for t, char in enumerate(name):
        try:
            X_t[i, t, char_indices[char]] = 1
        except:
            print("Не найден символ {} в имени: {}".format(char, name))

Не найден символ ' в имени: E'Lane


In [329]:
predicted = model.predict_classes(X_t)

3089/3089 [==============================] - 6s 2ms/step


In [339]:
X_tt = X_test[:,1].astype(float).astype(int)

In [340]:
metrics_test(predicted, X_tt)

acc=0.8330
micro F1=0.8330, micro P=0.8330, micro R=0.8330
macro F1=0.7666, macro P=0.7666, macro R=0.7666



# Выводы:

### 1) Нейронку обучать долго, но результаты лучше.
### 2) Чем больше размерность n-грамм, тем лучше результат.